# Comparison of results with and without optimization
- We compare costs, not full load hours. 
- We start with total costs first.
- We ignore transportation costs (as these are not part of the optimization).

## Data import and preparation

In [ ]:
# import libraries:

import plotly.express as px

import plotly.graph_objects as go
import pandas as pd

import ipywidgets as widgets

from IPython.display import display

import numpy as np


# import data from csv file:
df = pd.read_csv("cost_comparison.csv")
print(f"imported {len(df)} rows from csv.")

# remove reconversion chains:
# ind = ~df["chain"].str.contains("reconv. to H2")
# df = df.loc[ind]

# determine final product and electrolyzer type:
df[["final_product", "electrolyzer"]] = (
    df["chain"].str.strip(")").str.split(" \(", expand=True)
)

# remove subregions:
df = df.loc[~df["region"].str.endswith(")")]

# remove columns where all entries are identical:
for c in df.columns:
    if len(df[c].unique()) == 1:
        df = df.drop(c, axis=1)

print(f"After removing subregions,  {len(df)} rows remain.")

# aggregate over process types:
df_total = (
    df[
        [
            "scenario",
            "chain",
            "final_product",
            "electrolyzer",
            "res_gen",
            "region",
            "value_not_optimized",
            "value_optimized",
        ]
    ]
    .groupby(
        ["scenario", "chain", "final_product", "electrolyzer", "res_gen", "region"]
    )
    .sum()
    .reset_index()
)


print(f"After aggregating over cost categories, {len(df_total)} rows remain. ")

# calculating cost differences:
df_total["diff_abs"] = df_total["value_optimized"] - df_total["value_not_optimized"]
df_total["diff_rel"] = df_total["diff_abs"] / df_total["value_not_optimized"]


df_total.head()

In [ ]:
df["chain"].unique()

## Problem: Wind Offshore  and Wind Onshore in Jordan:
- Costs for Wind Offshore in Jordan is up to 350% higher than without optimization.
- Costs for Wind Onshore in Jordan are significantly lower than without optimization
- We remove these datapoints now and check on them later 

In [ ]:
# drop Jordan:
ind1 = df_total["region"] == "Jordan"
ind2 = df_total["res_gen"] == "Wind Offshore"
ind3 = df_total["res_gen"] == "Wind Onshore"
df_total_without_jordan = df_total.loc[~((ind1 & ind2) | (ind1 & ind3))]

## Problem with chains that include reconversion to H2
- TODO

We  create a function for plotting distribution of differences as box plots:

In [ ]:
def create_boxbplot(df: pd.DataFrame, y: str, x: str = "diff_rel"):
    df_plot = df.copy()
    medians = df_plot.groupby(y)[x].median()
    df_plot["median"] = np.where(
        df_plot[y].map(medians) > 0, "cost increase", "cost decrease"
    )

    df_plot = df_plot.sort_values("median", ascending=False)

    height = len(df_plot[y].unique()) * 75 + 100

    y_order = medians.sort_values(ascending=False).index.tolist()

    # fig = px.box(
    #     df_plot,
    #     x=x,
    #     y=y,
    #     color="median",
    #     hover_data=df_plot.columns.tolist(), height=height,
    #     category_orders={y:y_order},
    #     # points="all"
    # )

    fig = px.violin(
        df_plot,
        x=x,
        y=y,
        color="median",
        hover_data=df_plot.columns.tolist(),
        height=height,
        category_orders={y: y_order},
        box=True,
        # points="all"
    )

    fig.show()

## Data scenario:
- picture looks very similar across scenarios, so we can focus on one of them first
- overall: optimization leads to cost decrease
- effect is strongest in **low** scenarios. Why is that? Maybe lower storage costs?

In [ ]:
fig = create_boxbplot(df_total, "scenario")
fig = create_boxbplot(df_total_without_jordan, "scenario")

## RE source:
- PV: lower costs
- Wind: slightly lower costs
- Hybrid: higher costs
- check: extreme outliers with wind offshore (cost increase 350%)

In [ ]:
fig = create_boxbplot(df_total, "res_gen")
fig = create_boxbplot(df_total_without_jordan, "res_gen")

## Process chain:
- check: aha, the outliers all seem to be in jordan, Wind Offshore.
- why these strong cost decreases for chains with reconversion? Reconversion is not part of the optimization, there should be no differences at all!

In [ ]:
fig = create_boxbplot(df_total, y="chain")
fig = create_boxbplot(df_total_without_jordan, y="chain")

## Final product

In [ ]:
fig = create_boxbplot(df_total, y="final_product")
fig = create_boxbplot(df_total_without_jordan, y="final_product")

## Electrolyzer type:

In [ ]:
fig = create_boxbplot(df_total, y="electrolyzer")
fig = create_boxbplot(df_total_without_jordan, y="electrolyzer")

## Differences by region:
- no clear picture here. For the most regions, there is a surprisingly wide spread. 
- outlieres towards high cost increases seem to be almost always hybrid sites
- maybe we should redo this plot for each res-gen type separately. 

In [ ]:
# fig = create_boxbplot(df_total, y="region")
fig = create_boxbplot(df_total_without_jordan, y="region")

In [ ]:
df_hybrid = df_total_without_jordan.loc[
    df_total_without_jordan["res_gen"] == "Wind-PV-Hybrid"
]
create_boxbplot(df_hybrid, y="region")

In [ ]:
df_hybrid = df_total_without_jordan.loc[
    df_total_without_jordan["res_gen"] == "PV tilted"
]
create_boxbplot(df_hybrid, y="region")

In [ ]:
df_total["res_gen"].unique()